In [34]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [35]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio, math

In [36]:
iOption0 = 1
part = 0
N_loc_files = 2

In [37]:
N_centr = 5
N_pt = 1
#colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,1.5,1.0,2.0,0.5,5.0,1.0,5.0,0.5,5]
colors = [3,1,2,4,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_names=["../QA/input/NewHadron/missalign_hadron_new8_rg0-8.root","../sim/output/embed/piminus_10M_embed_hagedorn_v0_ddhit.root",\
            "../QA/input/AfterQM/my-m_ee_Run14AuAu_72nd_new_19730_993runs.root", "../sim/output/Newembed/deltas/250329/piminus_100M_v2.root"]
#file_names=[ "../sim/output/Newembed/deltas/250329/piminus_100M_v2.root"]
part_names = ["real","sim","#phi","J/#psi","#gamma#to ee"]
hist_dphi_names = ["dphi_hist_el_dynamic_2"]
hist_dthe_names = ["dthe_hist_el_dynamic_2"]
hist_sdphi_names=["sdphi_hist_el_dynamic_2"]
hist_sdthe_names=["sdthe_hist_el_dynamic_2"]
hists_all = [hist_dphi_names,hist_dthe_names,hist_sdphi_names,hist_sdthe_names]
N_type = len(hists_all)
names = ["#phi","#theta"]

In [38]:
hists_read = []

N_files = len(file_names)
for iFile in range(N_files):
    infile = root.TFile.Open(file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [39]:
legends, titles, leftline, rightline = [], [], [], []
def pair_DCA_draw(clock, x = 0.5, y = 1.5, phithe = 0, data = 0, iter = 0):
    ipt = 2
    type_loc =phithe
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",1400,450*1)
    c0.Divide(2,1)
    N_dca_files = 1
    for itype in range(1):
        max = 0

        if iter == 1 : 
            legends.append(root.TLegend(0.75,0.55,0.95,0.95))
            Format_Legend(legends[-1],0.075)
            titles.append(Draw_Legend_Title(0.2,0.85,0.55,0.95,"0-93% Au+Au",0.075))
        for iFile in range(0+data,N_dca_files+data):
            c0.cd(itype+1)
            #ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            #ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibinZ0,ibinZ1)
            project1= hists_read[iFile][type_loc][itype].Project3D("yx")
            project1.SetName(f"proj__{iFile}_{itype}_{clock}_{iter}")
            clr = colors[iFile]
            Format_Hist_total(project1, f"{names[phithe]} in L1, rad",  f"{names[phithe]} in L0, rad", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            if iFile == data :
                project1.Draw("colz")
                max = project1.GetMaximum()
            else:
                project1.Scale(max/project1.GetMaximum())          
                project1.Draw("H same")
            if iter == 1 :
                leftline.append(root.TF1(f"lefty_{iFile}_{itype}_{clock}_{iter}","((x<[1])?pow(2,20)*(x-[0]):pow(2,20)*(x-0.6))",-0.1,0.1))
                rightline.append(root.TF1(f"right_{iFile}_{itype}_{clock}_{iter}","pow(2,20)*(x-[0])",-0.1,0.1))
                Format_Func(leftline[-1], 3,root.kViolet,1,0.5)
                leftline[-1].SetFillColor(root.kViolet)
                leftline[-1].SetFillStyle(3001)
                Format_Func(rightline[-1],3,root.kViolet,1,0.5)
            leftline[-1].SetParameter(0,x)
            leftline[-1].SetParameter(1,y)
            leftline[-1].Draw("E3 same")
            rightline[-1].SetParameter(0,y)
            #rightline[-1].Draw("same")


            c0.cd(itype+2)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            clr = colors[iFile]
            Format_Hist_total(project, f"{names[phithe]}"+"_{L0}, rad",  f"dN/d{names[phithe]}, "+"rad^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            #project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
            #root.gPad.SetLogy()
            if iFile == data :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            if iter == 1 : legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[itype].Draw()
        titles[itype].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/dca2.png")

    return c0

In [40]:
def rebin_histogram_with_custom_bins(hist, center_range=(-0.01, 0.01), rebin_factor=11):
    """
    Rebin a histogram with custom binning:
    - Preserve original binning in the center range.
    - Rebin by a factor outside the center range.

    Parameters:
        hist (TH1): Original histogram.
        center_range (tuple): Range (min, max) where original binning is preserved.
        rebin_factor (int): Factor by which to reduce binning outside the center range.

    Returns:
        TH1F: New histogram with custom binning.
    """
    for ibin in range(hist.GetNbinsX()):
        bincenter = hist.GetBinCenter(ibin+1)
        if center_range[0] <= bincenter <= center_range[1]: continue
        newbincontent = 0
        for i in range(rebin_factor):
            newbincontent += hist.GetBinContent(max(1,min(ibin+1+i, hist.GetNbinsX())))
        hist.SetBinContent(ibin+1, newbincontent/rebin_factor)
        
    
    return hist

In [41]:
def my_log(hist):
    for ibinx in range(hist.GetNbinsX()):
        for ibiny in range(hist.GetNbinsY()):
            if hist.GetBinContent(ibinx+1,ibiny+1) > 0:
                hist.SetBinContent(ibinx+1,ibiny+1, math.log(hist.GetBinContent(ibinx+1,ibiny+1)))
    return hist

In [42]:
def smart_projection(hist,ibinZ0,ibinZ1,ibinT0,ibinT1, gap = 0.004):
    hist.GetZaxis().SetRange(ibinZ0,ibinZ1)
    new_hist = hist.Project3D("xy")
    for ibinx in range(new_hist.GetNbinsX()):
        for ibiny in range(new_hist.GetNbinsY()):
            if (abs(new_hist.GetXaxis().GetBinCenter(ibinx+1)-hist.GetYaxis().GetBinCenter(ibiny+1))>gap):
                new_hist.SetBinContent(ibinx+1,ibiny+1,0)
    return new_hist

In [43]:
kek = smart_projection(hists_read[0][0][0],1,-1,1,-1)
c2 = root.TCanvas("c2","c2",1400,450*1)
kek.Draw("colz")
c2.Draw()

In [44]:
# Define a soft gradient with pastel-like colors
stops = array('d', [0.0, 0.5, 1.0])  # Positions of colors
red = array('d', [0.8, 0.9, 1.0])    # Red component (light tones)
green = array('d', [0.9, 1.0, 0.8])  # Green component (soft green)
blue = array('d', [1.0, 0.9, 0.8])   # Blue component (light blue to pale yellow)


# Create the custom palette
root.TColor.CreateGradientColorTable(len(stops), stops, red, green, blue, 50)
root.gStyle.SetNumberContours(50)  

In [45]:
# Define a soft gradient inspired by Stony Brook colors with more contrast
stops = array('d', [0.0, 0.5, 1.0])  # Positions of colors
red = array('d', [0.8, 0.85, 1.0])   # Red component (more vibrant red for higher density)
green = array('d', [0.8, 0.75, 0.5]) # Green component (darker gray to soft red)
blue = array('d', [0.8, 0.75, 0.5])  # Blue component (darker gray to soft red)

# Create the custom palette
root.TColor.CreateGradientColorTable(len(stops), stops, red, green, blue, 50)
root.gStyle.SetNumberContours(50)  # Number of color levels

In [46]:
# Define a soft gradient inspired by Stony Brook colors with darker and deeper red
stops = array('d', [0.0, 0.7, 1.0])  # Positions of colors
red = array('d', [0.8, 0.9, 1.0])    # Red component (brighter red for higher density)
green = array('d', [0.8, 0.4, 0.0])  # Green component (reduced for more contrast)
blue = array('d', [0.8, 0.4, 0.0])   # Blue component (reduced for more contrast)

# Create the custom palette
root.TColor.CreateGradientColorTable(len(stops), stops, red, green, blue, 50)
root.gStyle.SetNumberContours(50)  # Number of color levels

In [47]:
legends, titles, leftline, rightline = [], [], [], []
def pair_DCA_draw_note(clock, x = 0.5, y = 1.5, phithe = 0, data = 0, iter = 0):
    ipt = 2
    type_loc =phithe
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",int(700/0.9),700)
    c0.Divide(1,1)
    N_dca_files = 1
    for itype in range(1):
        iFile = 3
        root.gPad.SetTopMargin(0.01)
        c0.cd(itype+1)
        legends.append(root.TLegend(0.75,0.55,0.95,0.95))
        project1= hists_read[iFile][type_loc][itype].Project3D("xy")
        project1.SetName(f"proj__{iFile}_{itype}_{clock}_{iter}")
        clr = colors[iFile]
        pxaxis = project1.GetXaxis()
        pyaxis = project1.GetYaxis()
        pxaxis.SetRangeUser(-0.059, 0.059)
        pyaxis.SetRangeUser(-0.059, 0.059)
        # Label only -0.03, 0, and 0.03 on x-axis and y-axis
        pxaxis.SetNdivisions(3, 5, 0)  # 3 labels: -0.03, 0, 0.03
        pyaxis.SetNdivisions(3, 5, 0)
        pxaxis.SetLabelSize(0.04)      # Adjust label size
        pyaxis.SetLabelSize(0.04)
        pxaxis.SetLabelOffset(0.01)   # Adjust label offset
        pyaxis.SetLabelOffset(0.01)
        #project1 = my_log(project1)
        
        Format_Hist_total(project1, f"{names[phithe]} in L1, rad",  f"{names[phithe]} in L0, rad", 0.15, 0.15,  0.11, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1., "",  1,  1.5)
        root.gPad.SetLogz()
        project1.SetMaximum(project1.GetMaximum()/1)
        project1.SetMinimum(project1.GetMinimum())
        #root.gStyle.SetPalette(root.kBird)
        project1.Draw("colz")          
        rect = root.TBox(-0.02, -0.02, 0.02, 0.02)  # Define rectangle coordinates
        rect.SetFillColorAlpha(root.kViolet, 0.4)  # Set violet color with 40% transparency
        rect.SetLineColor(root.kViolet)  # Set border color
        rect.SetLineWidth(2)  # Set border width
        rect.Draw("same")  # Draw the rectangle on the same pad
        continue
        
        c0.cd(itype+2)
        ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
        ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
        ibinT0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(-0.049)
        ibinT1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(+0.049)
        ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
        project_full = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_full{iter}",ibinT0,ibinT1,ibinZ0,ibinZ1)
        #project_full = smart_projection(hists_read[iFile][type_loc][itype],ibinZ0,ibinZ1,ibinT0,ibinT1).ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_mewfull{iter}",ibinT0,ibinT1)
        clr = colors[iFile]
        project = rebin_histogram_with_custom_bins(project, center_range=(-0.01, 0.01), rebin_factor=11)
        project_full.Smooth(1)
        project.Smooth(1)
        Format_Hist_total(project_full, f"{names[phithe]}"+"_{L0}, rad",  f"dN/d{names[phithe]}, "+"rad^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  3,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
        Format_Graph(project, Lcolor=root.kRed, Lwidth=3, Msize=0)
        project.SetLineStyle(1)
        #project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
        root.gPad.SetLogy()
        #project_full.Scale(1/project_full.Integral())
        #project.Scale(1/project.Integral())
        project_full.SetMaximum(project.GetMaximum()*5)
        project_full.SetMinimum(project.GetMinimum()/2)
        project_full.GetXaxis().SetRangeUser(-0.049, 0.049)
        project_full.Draw("H")
        project.Draw("S same")
        root.gStyle.SetOptStat(0)
        #legends[itype].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/dca2.png")

    return c0

In [48]:
if True:
    c = pair_DCA_draw_note(1,-0.001,+0.00,0,0,1)
    c.Draw()
    c.SaveAs(f"output/ddhit.pdf")

In [49]:
clock = 0

In [50]:

clock += 1

slider2 = widgets.FloatRangeSlider(
    value=[-0.05, 0.05],
    min=-0.099,
    max= 0.099,
    step=0.001,
    description='Mass [GeV]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    layout=widgets.Layout(width='75%'),
)

async def test2(slider2, output, phithe=0, data = 0):
    i = 0
    while slider2.value[0]< 0.09 and slider2.value[1] < 0.09:
        i = i+1
        await asyncio.sleep(1)
        output.update('phi in L1 ' + str(round(0.5*(slider2.value[0]+slider2.value[1]),3)) )
        c1= pair_DCA_draw(clock, slider2.value[0],slider2.value[1],phithe, data, i)
        if i == 1: c1.Draw()
        else : c1.Update()
    return

output = display("phi in L1: 0.00", display_id=True)
display(slider2)


task2 = asyncio.create_task(test2(slider2, output, 0, 0))

'phi in L1: 0.00'

FloatRangeSlider(value=(-0.05, 0.05), continuous_update=False, description='Mass [GeV]:', layout=Layout(width=…

In [51]:
if True: was_cancelled = task2.cancel()

In [52]:
#!jupyter nbconvert --to html sim_ddhit_assoc.ipynb --HTMLExporter.theme=dark --no-input

legends, titles, leftline, rightline = [], [], [], []
def pair_DCA_draw_note(clock, x = 0.5, y = 1.5, phithe = 0, data = 0, iter = 0):
    ipt = 2
    type_loc =phithe
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",1400,450*1)
    c0.Divide(2,1)
    N_dca_files = 1
    for itype in range(1):
        iFile = 0
        root.gPad.SetTopMargin(0.01)
        c0.cd(itype+1)
        legends.append(root.TLegend(0.75,0.55,0.95,0.95))
        project1= hists_read[iFile][type_loc][itype].Project3D("xy")
        project1.SetName(f"proj__{iFile}_{itype}_{clock}_{iter}")
        clr = colors[iFile]
        pxaxis = project1.GetXaxis()
        pyaxis = project1.GetYaxis()
        pxaxis.SetRangeUser(-0.049, 0.049)
        pyaxis.SetRangeUser(-0.049, 0.049)
        Format_Hist_total(project1, f"{names[phithe]} in L1, rad",  f"{names[phithe]} in L0, rad", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
        root.gPad.SetLogz()
        project1.Draw("col")          
        rect = root.TBox(-0.02, -0.02, 0.02, 0.02)  # Define rectangle coordinates
        rect.SetFillColorAlpha(root.kViolet, 0.4)  # Set violet color with 40% transparency
        rect.SetLineColor(root.kViolet)  # Set border color
        rect.SetLineWidth(2)  # Set border width
        rect.Draw("same")  # Draw the rectangle on the same pad

        
        c0.cd(itype+2)
        ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
        ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
        ibinT0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(-0.049)
        ibinT1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(+0.049)
        ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
        project_full = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X_full{iter}",ibinT0,ibinT1,ibinZ0,ibinZ1)
        clr = colors[iFile]
        project.S
        Format_Hist_total(project_full, f"{names[phithe]}"+"_{L0}, rad",  f"dN/d{names[phithe]}, "+"rad^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  3,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
        Format_Graph(project, Lcolor=root.kRed, Lwidth=3, Msize=0)
        project.SetLineStyle(1)
        #project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
        root.gPad.SetLogy()
        #project_full.Scale(1/project_full.Integral())
        #project.Scale(1/project.Integral())
        project_full.SetMaximum(project.GetMaximum()*5)
        project_full.SetMinimum(project.GetMinimum()/2)
        project_full.GetXaxis().SetRangeUser(-0.049, 0.049)
        project_full.Draw("H")
        project.Draw("S same")
        root.gStyle.SetOptStat(0)
        #legends[itype].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/dca2.png")

    return c0